


# AutoMashup Development Notebook

This notebook demonstrates an automated approach to creating music mashups. It leverages various libraries and techniques to analyze, process, and combine different musical tracks. The process involves:

1. **Preprocessing:** Extracting key features from input songs, such as beats, downbeats, segments, and key signatures using the `allin1` library.
2. **Track Selection:** Choosing specific parts (e.g., vocals, instrumentals) of the songs to be used in the mashup.
3. **Mashup Techniques:** Implementing algorithms to synchronize beats, adjust tempo and pitch, and align musical phases between the selected tracks using libraries like `pyrubberband` and `pyloudnorm`.
4. **Output:** Generating the final mashup audio file.

The notebook provides a flexible framework for experimenting with different mashup strategies and parameters. You can customize the input songs, track selections, and mashup techniques to create your unique mashups.

**Key Features:**

* Automated analysis and preprocessing of musical tracks.
* Flexible selection of song components for mashup creation.
* Multiple mashup techniques for beat synchronization, pitch adjustment, and phase alignment.
* Easy-to-use interface for experimentation and customization.


## Load folders and files
If you already saved some mashups in your drive, you can recover them so you don't have to process every song each time.

In [ ]:
import shutil
import os
from google.colab import drive
import zipfile

def load_project_from_drive(drive_path, archive_name="project_files.zip"):
  """Loads project files from a zip archive on Google Drive.

  Args:
    drive_path: The path on Google Drive where the archive is stored.
    archive_name: The name of the zip archive (default: "project_files.zip").
  """

  # 1. Mount Google Drive
  drive.mount('/content/drive')

  # 2. Construct the full path to the archive
  archive_path = os.path.join(drive_path, archive_name)

  # 3. Extract the archive
  with zipfile.ZipFile(archive_path, 'r') as zip_ref:
    zip_ref.extractall('.')  # Extract to the current working directory

  print(f"Project files loaded from '{archive_path}'")

# Example usage:
load_project_from_drive("/content/drive/My Drive/PathToProyect", archive_name="MashupList.zip")  # Replace with your drive path

In [ ]:
!pip install git+https://github.com/CPJKU/madmom  # install the latest madmom directly from GitHub
!pip install allin1  # install this package
!pip install natten==0.17.3
!pip install pydub
!pip install pymusickit
!pip install pyrubberband
!pip install pyloudnorm

  Cloning https://github.com/CPJKU/madmom to /tmp/pip-req-build-8lk8xnd5
  Running command git clone --filter=blob:none --quiet https://github.com/CPJKU/madmom /tmp/pip-req-build-8lk8xnd5
  Resolved https://github.com/CPJKU/madmom to commit 27f032e8947204902c675e5e341a3faf5dc86dae
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pyrubberband-0.4.0-py3-none-any.whl.metadata (1.2 kB)
Using cached pyrubberband-0.4.0-py3-none-any.whl (4.8 kB)


In [ ]:
!apt-get update
!apt-get install rubberband-cli

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,312 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,655 kB]
Get:13 https://r2u.stat.illinois.edu/u

## Preprocess a song

Before applying our mashup methods, we'll be getting some data out of the song we want to use.

For this notebook to be interesting, we suggest you to preprocess two songs first. (We suggest using .mp3 files over .wav)

In [ ]:
path = "Song1.mp3" # Path to the song
allin1.analyze(path, out_dir='./struct', demix_dir='./separated', keep_byproducts=True, overwrite=True)
key_finder(path)

### Loading the preprocessed songs as input for mashup

Once you preprocessed a song, it will be identified by a song name, which is the name of the file preprocessed, without the extension. For instance, for the song "./input/song.mp3", the track name is "song"

In [ ]:
# Put your song names here
song_name_1 = 'Song1'
song_name_2 = 'Song2'
# You will always have at least one track and up to four
# If you need one track as a reference (for instance for beat structure), you will use the first one

What we'll be calling a "track" is a python Track object with the following attributes :
  

* 'track_name' : String,
* 'audio' : audio of the track, it's a np array it can be only a part of the song (vocals, instru, ...),
* 'sr' : the sampling frequency,
* 'path' : the path of the original file
* 'bpm' : the bpm found by allin1 analysis
* 'beats' : beats list determined by allin1 analysis
* 'downbeats' : downbeats list determined by allin1 analysis
* 'segments' : list of the song's phases with their phase label (verse, chorus, ...)
* 'key' : correlation with each key (the highest value will be used as main key)
     

It's important that **your mashup methods returns the same kind of object !** Also, your method should handle up to 4 different tracks !
Especially, metadata should look like this :

    {
        "path": "/home/User/Documents/Automashup/AutoMashup/mashup/input/bazard\u00e9e.mp3",
        "bpm": 103,
        "beats": [
            4.06,
            4.63,
            5.23,
            5.81,
            ...
            ],
        "downbeats": [
            5.23,
            7.57,
            9.89,
            12.23,
            ...
        ],
        "beat_positions": [
            3,
            4,
            1,
            2,
            3,
            4,
            1,
            ...
        ],
        "segments": [
            {
            "start": 0.0,
            "end": 4.06,
            "label": "start"
            },
            {
            "start": 4.06,
            "end": 32.59,
            "label": "verse"
            },
            {
            "start": 32.59,
            "end": 51.21,
            "label": "chorus"
            },
            ...
        ],
        "key": {
            "C major": -0.512,
            "C# major": 0.29,
            ...
        }
    }


Do not freak out ! Metadatas are created within the analysis (within each track) so you won't have to set everything by hand. However, try to modify the metadatas according to the modification you do to a track to keep them consistent along the process !

##Utils

In [ ]:
import numpy as np
import os
import shutil
import math
import json
import soundfile as sf
from pymusickit.key_finder import KeyFinder
from collections import OrderedDict
from pydub import AudioSegment
from pydub.playback import play

### Here are some useful functions used in other parts of the project


def increase_array_size(arr, new_size):
    if len(arr) < new_size:
        # Create a new array with the new size
        increased_arr = np.zeros(new_size)
        # Copy elements from the original array to the new array
        increased_arr[:len(arr)] = arr
        return increased_arr
    else:
        return arr


import os

def get_path(track_name, type):
    # returns the path of a song
    # It can return the whole track or just a separated part of it,
    # depending on the type, which should be one of the following :
    # 'entire', 'bass', 'drums', 'vocals', 'other'

    # Extract the filename without extension
    track_name_no_ext = os.path.splitext(track_name)[0]

    if type == 'entire':
        if os.path.exists('./input/' + track_name):
            path = './input/' + track_name
        else:
            path = './input/' + track_name_no_ext + '.wav' # Check for .wav if .mp3 is not found
            if not os.path.exists(path):
                path = './input/' + track_name_no_ext + '.mp3' # Check for .mp3 if .wav is not found
    else:
        path = './separated/htdemucs/' + track_name_no_ext + '/' + type + '.wav'
        if not os.path.exists(path):
            path = './separated/htdemucs/' + track_name_no_ext + '/' + type + '.mp3'
            if not os.path.exists(path): # Check for common extension mismatches
                path = './separated/htdemucs/' + track_name_no_ext + '/' + type + '.wav'
                if not os.path.exists(path):
                    path = './separated/htdemucs/' + track_name_no_ext + '/' + type + '.mp3'
    assert(os.path.exists(path)), f"File not found: {path}" # Added a more informative error message
    return path


def remove_track(track_name):
    # function to remove a track and all the files that concern it.
    struct_path = "./struct/" + track_name + ".json"
    folder_path = "./separated/htdemucs/" + track_name + "/"
    os.remove(struct_path)
    shutil.rmtree(folder_path)


def extract_filename(file_path):
    # Extract filename from a given path
    filename = os.path.basename(file_path)
    filename_without_extension, _ = os.path.splitext(filename)
    return filename_without_extension


def note_to_frequency(key):
    # turn a note with a mode to a frequency
    note, mode = key.split(' ', 1)
    reference_frequency=440.0
    semitone_offsets = {'C': -9, 'C#': -8, 'Db': -8, 'D': -7, 'D#': -6, 'Eb': -6, 'E': -5, 'Fb': -5, 'E#': -4,
                        'F': -4, 'F#': -3, 'Gb': -3, 'G': -2, 'G#': -1, 'Ab': -1, 'A': 0, 'A#': 1, 'Bb': 1, 'B': 2, 'Cb': 2, 'B#': 3}
    semitone_offset = semitone_offsets[note]
    if mode == 'minor':
        semitone_offset -= 3
    frequency = reference_frequency * 2 ** (semitone_offset / 12)
    return frequency


def key_finder(path):
    filename = extract_filename(path)
    struct_path = f"./struct/{filename}.json"
    with open(struct_path, 'r') as file:
        data = json.load(file)
        data['key'] = KeyFinder(path).key_dict
    with open(struct_path, 'w') as file:
        json.dump(data, file, indent=2)


def calculate_pitch_shift(source_freq, target_freq):
    pitch_shift = 12 * math.log2(target_freq / source_freq)
    return pitch_shift


def key_from_dict(dict):
    # get key from the metadata correlation list
    best_key, best_score = "", ""
    for key, score in dict.items():
        if best_score=="" or best_score<score:
            best_key, best_score = key, score
    return best_key


def closest_index(value, value_list):
    # get the index of the closest value of a specific target in a list
    closest_index = min(range(len(value_list)), key=lambda i: abs(value_list[i] - value))
    return closest_index

def get_unique_ordered_list(input_list):
    # Use OrderedDict to preserve order while removing duplicates
    return list(OrderedDict.fromkeys(input_list))

# extracts the start and end times for the selected segment
def get_segment_times(segments, selected_label):
    for segment in segments:
        if segment['label'] == selected_label:
            return segment['start'], segment['end']
    return 0, 2  # Default to first 2 seconds if segment not found

def extract_audio_segment(file_path, start_time, end_time, output_path):
    try:
        # Load the audio file
        audio = AudioSegment.from_file(file_path)
        # Extract the segment
        segment = audio[start_time * 1000:end_time * 1000]  # pydub works in milliseconds
        # Export the segment to a temporary file
        segment.export(output_path, format="mp3")
        return output_path
    except Exception as e:
        print(f"Error extracting audio segment: {e}")
        return None

# Function to merge the segments that have the same label and are next to each other
def merge_segments(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    merged_segments = []
    previous_segment = None

    for segment in data.get('segments', []):
        if previous_segment and previous_segment['label'] == segment['label']:
            # Merge with the previous segment
            previous_segment['end'] = segment['end']
        else:
            # Add the previous segment to the list if it exists
            if previous_segment:
                merged_segments.append(previous_segment)
            # Update the previous segment
            previous_segment = segment

    # Add the last segment
    if previous_segment:
        merged_segments.append(previous_segment)

    # Update the segments in the data
    data['segments'] = merged_segments

    # Save the modified data back to the JSON file
    with open(json_path, 'w') as f:
        json.dump(data, f, indent=2)

##Segment


In [ ]:
import numpy as np
import copy
import pyrubberband as pyrb

# Define a Track class to represent a part of a track
# The aim of this kind of object is to keep together the audio itself,
# and the beats

class Segment:

    transition_time = 0.5 # transition time in seconds

    def __init__(self, segment_dict):
        # We create a segment from a dict coming from metadata.
        # They look like this :
        # {
        #   "start": 0.4,
        #   "end": 22.82,
        #   "label": "verse"
        # }


        for key in segment_dict.keys():
            setattr(self, key, segment_dict[key])

        # We calculate the time in minutes for each segment
        self.duration = (self.end - self.start)/60



    def link_track(self, track):
        # Function to link a segment to a track, must be set whenever
        # we create a segment
        # it loads all the pieces of information useful for a segment
        self.sr = track.sr
        beats = track.beats
        downbeats = track.downbeats


        start_beat = closest_index(self.start, beats)
        end_beat = closest_index(self.end, beats)

        self.beats = beats[start_beat:end_beat]

        if self.beats == []:
            self.downbeats = []
            self.audio = np.array([])
            self.left_transition = np.array([])
            self.right_transition = np.array([])

        else:
            # Make sure the first beat and downbeat starts on zero
            self.beats = self.beats - np.repeat(self.beats[0], len(self.beats))

            self.downbeats = downbeats - np.repeat(downbeats[0], len(downbeats))
            self.downbeats = [downbeat for downbeat in self.downbeats if downbeat in self.beats]
            if self.downbeats != []:
                self.downbeats = self.downbeats - np.repeat(self.downbeats[0], len(self.downbeats))

            self.audio = track.audio[round(self.start*self.sr):round(self.end*self.sr)]

            if self.start - self.transition_time > 0 :
                self.left_transition = track.audio[round((self.start-self.transition_time)*self.sr):round(self.start*self.sr)]
            else :
                self.left_transition = track.audio[:round(self.start*self.sr)]
            if self.end + self.transition_time < len(track.audio) * self.sr:
                self.right_transition = track.audio[round(self.end*self.sr):round((self.end+self.transition_time)*self.sr)]
            else :
                self.right_transition = track.audio[round(self.end*self.sr):]

    def concatenate(self):
        # Calculate the seconds to shift the incoming beats to start where the current segment's audio ends.
        offset = len(self.audio) / self.sr

        # Concatenate the beats array of the current segment
        new_beats = self.beats + offset
        self.beats = np.concatenate([self.beats, new_beats])

        # Concatenate the downbeats array
        new_downbeats = self.downbeats + offset
        self.downbeats = np.concatenate([self.downbeats, new_downbeats])

        # Concatenate the audio data of the two segments
        self.audio = np.concatenate((self.audio, self.audio))


    def get_audio_beat_fitted(self, beat_number, tempo, duration, sr):
        """
        Adjusts the audio segment to fit a specified number of beats at a given tempo.

        Parameters:
        - beat_number: The target number of beats for the segment.
        - tempo: The target tempo (BPM) for the segment.
        - duration: The target segment duration

        Returns:
        - A new audio segment adjusted to the specified number of beats and tempo.
        """
        try:
            # Make a deep copy of the segment to avoid modifying the original
            result = copy.deepcopy(self)

            if beat_number == 0:
                # If beat_number is 0, return an empty segment
                result.audio = np.array([])
                result.beats = []
                result.downbeats = []
            else:
                # We compare the bpm of the target segment and the current segment.
                # If the difference is too big, half or double it
                segment_bpm =(len(result.beats)/result.duration)
                if tempo / segment_bpm > 1.5:
                    tempo /= 2
                    beat_number //= 2
                    # If we reduce the bpm to half, then the amount of beats as well so the duration stays the same
                elif tempo / segment_bpm < 0.75:
                    tempo *= 2
                    beat_number //= 2
                else:
                    pass

                # We calculate the rate of stretch for the segment.
                stretch_rate = tempo / segment_bpm
                result.audio = pyrb.time_stretch(result.audio, sr, rate=stretch_rate)

                # We concatenate the segment to itself if it's shorter than the target
                while len(result.beats) < beat_number:
                    print(f"Concatenating for segment '{result.label}'")
                    result.concatenate()

                # Then we cut the amount of beats and downbeats so they end at the same time
                result.beats = result.beats[:beat_number]
                result.downbeats = [downbeat for downbeat in result.downbeats if downbeat <= result.beats[-1]]

                # Then we cut the audio for the same length as the objetive
                result.audio = result.audio[:duration]

            return result

        except Exception as e:
            # Handle any exceptions gracefully and print detailed error message
            print(f"Error fitting segment '{self.label}' to {beat_number} beats: {e}")
            raise e  # or return None or handle the error appropriately

##Track

In [ ]:
import json
import librosa
import numpy as np
import pyrubberband as pyrb



# Define a Track class to represent a musical track
# This class uses objects of type Segment
# The aim of this kind of object is to keep together the audio itself,
# the sampling frequency and the metadata coming from allin1 analysis

class Track:
    transition_time = 1 # transition time in seconds

    # Standard constructor
    def __init__(self, track_name, audio, metadata, sr):

        # Initialize track properties
        self.name = track_name
        self.audio = audio
        self.sr = sr
        self.segments = []

        # Load track metadatas
        for key in metadata.keys():
            if key!="segments":
                setattr(self, key, metadata[key])
            else:
                for segment in metadata["segments"]:
                    # Create segment objects
                    if isinstance(segment, Segment):
                        segment_ = segment
                    else:
                        # we handle the segments as stored in the struct files
                        segment_ = Segment(segment)
                    segment_.link_track(self)
                    self.segments.append(segment_)


    def track_from_song(track_name, type):
        # Function to create a track from a preprocessed song
        # type should be one of the following :
        # 'entire', 'bass', 'drums', 'vocals', 'other'
        name = track_name + ' - ' + type
        audio, sr = librosa.load(get_path(track_name, type), sr=None)
        struct_path = f"./struct/{track_name}.json"
        with open(struct_path, 'r') as file:
            metadata = json.load(file)
        return Track(name, audio, metadata, sr)

    # Must use the key from the voice
    def get_key(self):
        # Function to retrieve the key of a track
        # we use the "key" metadata which is a list of correlation
        # with each key. We look for the max correlation to get the
        # right key
        best_key, best_score = "", ""
        for key, score in self.key.items():
            if best_score=="" or best_score<score:
                best_key, best_score = key, score
        return best_key


    def __repitch(self, semitone_shift):
        # Function to repitch a track using a semitone shift
        # https://www.youtube.com/watch?v=Y2lUmwB7lzI
        shifted_audio = pyrb.pitch_shift(y=self.audio, sr=self.sr, n_steps=semitone_shift)
        self.audio = shifted_audio


    def pitch_track(self, target_key):
        # Function to repitch a track to a target key
        target_frequency = note_to_frequency(target_key)
        track_frequency = note_to_frequency(self.get_key())
        self.__repitch(calculate_pitch_shift(track_frequency, target_frequency))


    def add_metronome(self):
        # Function to add metronome sounds on the beats according
        # to the metadata of the track sounds
        downbeat_sound_audio, _ = librosa.load("../metronome-sounds/block.mp3")
        otherbeat_sound_audio, _ = librosa.load("../metronome-sounds/drumstick.mp3")

        # add sound for each beat
        for i, beat_frame in enumerate(self.beats):
            # if it's a downbeat, use the according sound
            clic_sound = downbeat_sound_audio if i % 4 == 0 else otherbeat_sound_audio
            clic = increase_array_size(clic_sound, len(self.audio[round(self.sr*beat_frame):]))
            # check that we do not get out of the track's bounds
            if len(self.audio[round(self.sr*beat_frame):])>=len(clic):
                self.audio[round(self.sr*beat_frame):] += clic


    def fit_phase(self, target_track):
        # Function to align track phases (verse, chorus, bridge, ...)
        # to a target track.
        # we'll do loops on the track to reach the number of beats targetted
        # for each phase
        # The challenge for this function is to keep the beats metadata
        # updated
        audio = np.array([])

        # lists of the return track beats and downbeats
        # we put 0 for convenience (see beats[-1] after)
        beats = [0]
        downbeats = [0]


        print(f" ********************** Adjusting the song {self.name}  **********************")

        # List of already found segments
        found_segments = {}

        # loop over each phase to reproduce
        for target_segment in target_track.segments:
            i = 0
            found_segment = False
            current_label = target_segment.label

            # Check if we have already found this label before
            if current_label in found_segments:
                start_index = found_segments[current_label] + 1
            else:
                start_index = 0

            i = start_index

            # loop over each segment to find occurrences
            while i < len(self.segments):
                segment = self.segments[i]
                if segment.label == current_label:
                    # If this is the first time we find the label or we have moved past the previous index
                    if not found_segment or i > found_segments[current_label]:
                        found_segment = True
                        tempo = round(len(segment.beats) / segment.duration)
                        found_segments[current_label] = i
                        break
                i += 1

            # If no segment was found, reuse the last found segment
            if not found_segment and current_label in found_segments:
                last_found_index = found_segments[current_label]
                segment = self.segments[last_found_index]
                tempo = round(len(segment.beats) / segment.duration)
                found_segment = True

            # if we do not find it, we add zeros with the right length
            if (not found_segment):
                tempo = round(len(target_segment.beats)/target_segment.duration)
                try:
                    if tempo == 0:
                        segment_length = 0
                    else:
                        segment_length = int((len(target_segment.beats) / (tempo / 60) * self.sr))
                    audio = np.concatenate([audio, np.zeros(segment_length)])
                    beats += [beats[-1] + (i + 1) / (tempo / 60) for i in range(len(target_segment.beats))]
                    downbeats += [downbeats[-1] + (4 * i + 1) / (tempo / 60) for i in range(len(target_segment.beats) // 4)]
                except Exception as e:
                    print(f"Error fitting silence. Error: {e}")
            else:
                try:
                    # if we find it, we make it fit to the desired beat number
                    if len(target_segment.beats) > 0:
                        target_bpm = len(target_segment.beats)/target_segment.duration

                        segment_fitted = segment.get_audio_beat_fitted(len(target_segment.beats), target_bpm, len(target_segment.audio), self.sr)
                        audio = np.concatenate([audio, segment_fitted.audio])

                        # reset first beat position per segment
                        track_sr = target_track.sr
                        track_beginning_temporal = target_segment.beats[0]
                        track_beginning = track_beginning_temporal * track_sr
                        # reset first beat position
                        audio = np.array(audio)[round(track_beginning):]

                        # we add the new beats to be able to sync after
                        beats += [beats[-1] + phase_beat for phase_beat in segment_fitted.beats]
                        downbeats += [downbeats[-1] + phase_downbeat for phase_downbeat in segment_fitted.downbeats]
                    # If its empty we ignore it
                    else: pass

                except Exception as e:
                    print(f"Error fitting segment: {segment.label} at {segment.start}, Error: {e}")
                    continue

        # we get rid of the first beats added for convenience
        beats = beats[1:]
        downbeats = downbeats[1:]

        self.audio = audio
        self.beats = beats
        self.downbeats = downbeats

    def get_segments(track_name):
        # This method should return a list of segments for the given song
        track = Track.track_from_song(track_name, 'entire')
        return [segment.label for segment in track.segments]

    def get_segments_full(track_name):
        # This method should return a list of the structure of segments for the given song
        track = Track.track_from_song(track_name, 'entire')
        return [{'start': segment.start, 'end': segment.end, 'label': segment.label} for segment in track.segments]

# Choose the sounds of each song

In [ ]:
## Let's create some Track objects with our preprocessed songs
# We'll be loading the vocals of the first song and the whole instru
# of the second song

tracks =  [] # input of the mashup methods

# type attribute enables to choose a separated part of a song (from demucs source separation)
# it can be 'vocals', 'bass', 'drums' or 'other'
track_1 = Track.track_from_song(song_name_1, type='vocals')
track_2 = Track.track_from_song(song_name_2, type='bass')
track_3 = Track.track_from_song(song_name_2, type='drums')
track_4 = Track.track_from_song(song_name_2, type='other')

tracks = [track_1, track_2, track_3, track_4]

In [ ]:
# We can have access to some attributes :
print(f"BPM : {track_1.bpm}")
print(f"Key correlation : {track_1.key}")
print(f"Beat frames : {track_1.beats}")
print(f"Track audio : {track_1.audio}")
print(f"Track Sampling Frequency {track_1.sr}")

#MASHUP
Define all mashup techniques

In [ ]:
import librosa
import numpy as np
import pyrubberband as pyrb
import pyloudnorm as pyln

# Mashup Technics
# In this file, you may add mashup technics
# the input of such a method is a list of up to 4 objects of type Track.
# You can modify them without making any copy, it's already done before.
# You may find useful methods in the track.py file
# Be sure to return a Track object

def mashup_technic(tracks, phase_fit=False, target_loudness=-14.0):
    # Mashup technic with first beat alignment and bpm sync
    sr = tracks[0].sr # The first track is used to determine the target bpm
    tempo = tracks[0].bpm
    main_track_length = len(tracks[0].audio)
    beginning_instant = tracks[0].beats[0] # beats metadata
    beginning = beginning_instant * sr
    mashup = np.zeros(0)
    mashup_name = ""

    # we add each track to the mashup
    for track in tracks:
        mashup_name += track.name + " " # name
        track_tempo = track.bpm
        if track == tracks[0]:
            track_beginning_temporal = track.beats[0]
        else:
            track_beginning_temporal = track.downbeats[0]
        track_sr = track.sr
        track_beginning = track_beginning_temporal * track_sr
        track_audio = track.audio

        # reset first beat position
        track_audio_no_offset = np.array(track_audio)[round(track_beginning):]

        # Change the bpm if there is no phase fit
        if not phase_fit:
            track_audio_accelerated = pyrb.time_stretch(track_audio_no_offset, track_sr, rate = tempo / track_tempo)
        else:
            #bpm is handled in segment.py
            track_audio_accelerated = track_audio_no_offset

        # add the right number of zeros to align with the main track
        final_track_audio = np.concatenate((np.zeros(round(beginning)), track_audio_accelerated))

        size = max(len(mashup), len(final_track_audio))
        mashup = np.array(mashup)
        mashup = (increase_array_size(final_track_audio, size) + increase_array_size(mashup, size))

    # Adjust mashup length to be the same as the main track's audio length
    if len(mashup) > main_track_length:
        mashup = mashup[:main_track_length]
    else:
        mashup = increase_array_size(mashup, main_track_length)

    # Apply LUFS normalization to the mashup to modify the loudness of the result
    meter = pyln.Meter(sr)  # Create a BS.1770 loudness meter
    mashup_loudness = meter.integrated_loudness(mashup)  # Measure the loudness
    print(f"Mashup loudness (before normalization): {mashup_loudness} LUFS")

    # Normalize the mashup to the target loudness (-14 LUFS by deafult)
    mashup_normalized = pyln.normalize.loudness(mashup, mashup_loudness, target_loudness)
    print(f"Mashup normalized to {target_loudness} LUFS")

    # we return a modified version of the first track
    # doing so, we keep its metadata
    tracks[0].audio = mashup

    return tracks[0]


def mashup_technic_repitch(tracks):
    # Mashup technique to change the key by repitch
    key = tracks[0].get_key() # target key
    for i in range(len(tracks)-1):
        tracks[i+1].pitch_track(key) # repitch

    return mashup_technic(tracks)


def mashup_technic_fit_phase(tracks):
    # Mashup technique with phase alignment (i.e., chorus with chorus, verse with verse...)
    # Each track's structure is aligned with the first one
    for i in range(len(tracks) - 1):
        tracks[i + 1].fit_phase(tracks[0])

    # Standard mashup method
    return mashup_technic(tracks, phase_fit=True)

def mashup_technic_fit_phase_repitch(tracks):
    # Mashup technique with phase alignment and repitch
    # Repitch has to be the last method for it to be effective
    key = tracks[0].get_key() # target key
    for i in range(len(tracks)-1):
        tracks[i + 1].fit_phase(tracks[0]) # phase fit
        tracks[i+1].pitch_track(key) # repitch
    # Phase fit mashup
    return mashup_technic(tracks, phase_fit=True)

#Choose the Mashup Technic you want to apply to your mashup

In [ ]:
### Apply the method here
mashup_result = mashup_technic_fit_phase_repitch(tracks) # Apply the mashup_technic function to the 'tracks' list.
# Changed 'mashup' to 'mashup_result' to store results and avoid overwriting module name

In [ ]:
# Have a listen
Audio(mashup_result.audio, rate = mashup_result.sr)

In [ ]:
# Save the file :
sf.write("mashup.mp3", mashup_result.audio, mashup_result.sr)

## Save folders and files

In [ ]:
import shutil
import os
from google.colab import drive
import zipfile

def save_folders_to_drive(folders_to_save, drive_path, archive_name="project_files.zip"):
  """Saves specified folders to Google Drive as a zip archive.

  Args:
    folders_to_save: A list of folder paths to save.
    drive_path: The desired path on Google Drive to save the archive.
    archive_name: The name of the zip archive (default: "project_files.zip").
  """

  # 1. Mount Google Drive
  drive.mount('/content/drive')

  # 2. Create the target directory if it doesn't exist
  os.makedirs(drive_path, exist_ok=True)

  # 3. Create the zip archive and add folders to it
  drive_save_path = os.path.join(drive_path, archive_name)
  with zipfile.ZipFile(archive_name, 'w') as zipf:
      for folder in folders_to_save:
          for root, dirs, files in os.walk(folder):
              for file in files:
                  zipf.write(os.path.join(root, file),
                             os.path.relpath(os.path.join(root, file),
                                             os.path.join(folder, '..')))

  # 4. Copy the archive to Google Drive
  shutil.copy(archive_name, drive_save_path)
  print(f"Folders saved to Google Drive at '{drive_save_path}'")

# Example usage:
#folders_to_save = ['./input', './separated', './struct', './spec']
#save_folders_to_drive(folders_to_save, "/content/drive/My Drive/Path")

We verify all uploaded songs from drive have their metadata

In [ ]:
from google.colab import drive
from os import listdir, path

drive.mount('/content/drive')

songs = listdir("/content/drive/My Drive/PathToProyect/Music")
print(songs)

# Loop to analyze each file
for i in range(len(songs)):
    file = songs[i]

    # We extract the filename without the extension
    name_wo_extension = path.splitext(file)[0]

    # Verification if the .json file already exists in struct
    file_json = name_wo_extension + ".json"
    if file_json not in listdir('./struct'):
        # Creates the full path of the file
        path_file = "/content/drive/My Drive/PathToProyect/" + file

        # Launching the analysis if the .json file does not exist in struct
        allin1.analyze(path_file, out_dir='./struct', demix_dir='./separated', keep_byproducts=True, overwrite=True)

        # Launching the key_finder function
        key_finder(path_file)
    else:
        print(f"The file {file_json} already exist in './struct'.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['Mia Doi Todd - I gave you my home.mp3', 'Dragon Or Emperor - Part of Me Says.mp3', 'The J. Arthur Keenes Band - The Boring World of Niels Bohr.mp3', 'The Oranges Band - Ride the Nuclear Wave.mp3', 'Bam Bam - Hi-Q.mp3', 'Shearer - Itch.mp3', 'Grand Mal - Children of Light.mp3', 'Poland - Lying Machine.mp3', 'Los Steaks - Sunday Girls.mp3', 'The Cute Lepers - Young Hearts.mp3', 'The Embarrassment - Patio Set.mp3', 'Short Hand - Certain Strangers.mp3', 'Home Blitz - Secret Wave.mp3', 'David Rovics - We Just Want The World.mp3', 'ZOE.LEELA - Jewel.mp3', 'deef - Ein sonniger Tag mit dir.mp3', 'Orange Peels - Grey Holiday.mp3', 'Josh Woodward - Cien Volando.mp3', 'Wann - Happy Birthday.mp3', 'Bessie Smith - My Sweetie Went Away.mp3', 'Benji Cossa - New Flowers (Fast 4-track Version).mp3']
Le fichier Mia Doi Todd - I gave you my home.json existe déjà dans './struc

In [ ]:
#Save in drive in Mashup
folders_to_save = ['./input', './separated', './struct', './spec']  # Added './spec'
save_folders_to_drive(folders_to_save, "/content/drive/My Drive/PathToProyect", archive_name="mashup2.zip")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Folders saved to Google Drive at '/content/drive/My Drive/ProCom/Processed/mashup2.zip'
